# TPS May 2022 - Scikit-Learn Ensemble

The competition description says this is a linear regression problem, so lets see what Scikit Learn can do
- https://scikit-learn.org/stable/modules/linear_model.html#linear-model

In [ ]:
import numpy as np 
import pandas as pd
import sklearn
from operator import itemgetter
from sklearn.linear_model import *
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from markupsafe import escape
import pprint

pp = pprint.PrettyPrinter(width=41, compact=True)

In [ ]:
%%time
col_dtypes = {
  "f_00": "float32",
  "f_01": "float32",
  "f_02": "float32",
  "f_03": "float32",
  "f_04": "float32",
  "f_05": "float32",
  "f_06": "float32",
  "f_07": "int32",
  "f_08": "int32",
  "f_09": "int32",
  "f_10": "int32",
  "f_11": "int32",
  "f_12": "int32",
  "f_13": "int32",
  "f_14": "int32",
  "f_15": "int32",
  "f_16": "int32",
  "f_17": "int32",
  "f_18": "int32",
  "f_19": "float32",
  "f_20": "float32",
  "f_21": "float32",
  "f_22": "float32",
  "f_23": "float32",
  "f_24": "float32",
  "f_25": "float32",
  "f_26": "float32",
  "f_27": "category",
  "f_28": "float32",
  "f_29": "int32",
  "f_30": "int32",
  "target": "int32",
}
def preprocess_df(df):
    # Split f_27 into letters
    col_names = ['f_27_0','f_27_1','f_27_2','f_27_3','f_27_4','f_27_5','f_27_6','f_27_7','f_27_8','f_27_9','f_27_10','f_27_00']
    df[col_names] = df['f_27'].str.split('',expand=True).astype("category")
    del df['f_27']
    del df['f_27_0']   # split gives emoty columns on end
    del df['f_27_00']  # split gives emoty columns on end
    
    # One Hot Encode
    for col_name in col_names:
        if col_name not in df.columns: continue 
        try:
            dummies = pd.get_dummies(df[col_name], prefix=col_name, drop_first=False)
            df = pd.concat([df, dummies], axis=1)
        except: continue
        del df[col_name]                        
    return df

def fix_missing_columns(train_df, test_df):
    missing_cols = (set(train_df.columns) - set(test_df.columns))  \
                 | (set(test_df.columns)  - set(train_df.columns)) \
                 - set(["target"])
    for col in missing_cols:
        train_df[col] = train_df.get(col,0)  # add zeros column if missing
        test_df[col]  = test_df.get(col,0)   # add zeros column if missing
    return train_df, test_df
    

train_df = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv', index_col='id', dtype=col_dtypes)
test_df  = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv',  index_col='id', dtype=col_dtypes)
train_df = preprocess_df(train_df)
test_df  = preprocess_df(test_df)
train_df, test_df = fix_missing_columns(train_df, test_df)

columns = test_df.columns
X       = train_df[columns]
Y       = train_df['target']
X_train, X_valid, Y_train, Y_valid = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)
X_test  = test_df[columns]

display('train_df')
display( train_df.info(verbose=True, memory_usage="deep") )
display( train_df )
display('test_df')
# display( test_df.info(verbose=True, memory_usage="deep") )
display( test_df )

# Preprocessing

Adding PolynomialFeatures actually reduces the score, so this is indeed a pure linear regression problem
- `degree=2` = `0.76347`
- `degree=1` = `0.72935`

In [ ]:
# DOCS: https://scikit-learn.org/stable/modules/preprocessing.html
# DOCS: https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions
def preprocess_X(X, degree=1):
    # NOTE: PolynomialFeatures needs to be done before scaling - https://towardsdatascience.com/preprocessing-with-sklearn-a-complete-and-comprehensive-guide-670cb98fcfb9
    X = PolynomialFeatures(degree=degree).fit_transform(X)
    X = StandardScaler().fit_transform(X)
    return X

columns = test_df.columns
X_test  = preprocess_X( test_df[columns],  degree=2 )
X       = preprocess_X( train_df[columns], degree=2 )
Y       = train_df['target']
X_train, X_valid, Y_train, Y_valid = sklearn.model_selection.train_test_split(X, Y, test_size=0.05, random_state=42)

print('X_train.shape', X_train.shape)
print('Y_train.shape', Y_train.shape)

# Linear Models

Lets try out each of the Linear Models from scikit-learn in a loop
- https://scikit-learn.org/0.20/modules/classes.html#module-sklearn.linear_model

In [ ]:
# Lazy Load models to avoid out of memory errors
models = [
    (sklearn.linear_model.ARDRegression, {}),          
    # ([n_iter, tol, …])	Bayesian ARD regression.
    
    (sklearn.linear_model.BayesianRidge, {}),          
    # ([n_iter, tol, …])	Bayesian ridge regression
    
    (sklearn.linear_model.ElasticNet, {}),             
    # ([alpha, l1_ratio, …])	Linear regression with combined L1 and L2 priors as regularizer.
    
    # (sklearn.linear_model.ElasticNetCV, {"max_iter":10_000}),           
    # ([l1_ratio, eps, …])	Elastic Net model with iterative fitting along a regularization path.
    
    (sklearn.linear_model.HuberRegressor, {"max_iter":1000}),         
    # ([epsilon, …])	Linear regression model that is robust to outliers.
    
    (sklearn.linear_model.Lars, {}),                   
    # ([fit_intercept, verbose, …])	Least Angle Regression model a.k.a.
    
    (sklearn.linear_model.LarsCV, {}),                 
    # ([fit_intercept, …])	Cross-validated Least Angle Regression model.
    
    (sklearn.linear_model.Lasso, {}),                  
    # ([alpha, fit_intercept, …])	Linear Model trained with L1 prior as regularizer (aka the Lasso)
    
    (sklearn.linear_model.LassoCV, {"max_iter":10_000}), 
    # ([eps, n_alphas, …])	Lasso linear model with iterative fitting along a regularization path.
    
    (sklearn.linear_model.LassoLars, {}),              
    # ([alpha, …])	Lasso model fit with Least Angle Regression a.k.a.
    
    (sklearn.linear_model.LassoLarsCV, {}),            
    # ([fit_intercept, …])	Cross-validated Lasso, using the LARS algorithm.
    
    (sklearn.linear_model.LassoLarsIC, {}),            
    # ([criterion, …])	Lasso model fit with Lars using BIC or AIC for model selection
    
    (sklearn.linear_model.LinearRegression, {}),       
    # ([…])	Ordinary least squares Linear Regression.  
    
    # (sklearn.linear_model.LogisticRegression, {}),     
    # ([penalty, …])	Logistic Regression (aka logit, MaxEnt) classifier.
    
    # (sklearn.linear_model.LogisticRegressionCV, {}),   
    # ([Cs, …])	Logistic Regression CV (aka logit, MaxEnt) classifier.
    
    # (sklearn.linear_model.MultiTaskLasso, {}),         
    # ([alpha, …])	Multi-task Lasso model trained with L1/L2 mixed-norm as regularizer.
    
    # (sklearn.linear_model.MultiTaskElasticNet, {}),    
    # ([alpha, …])	Multi-task ElasticNet model trained with L1/L2 mixed-norm as regularizer
    
    # (sklearn.linear_model.MultiTaskLassoCV, {}),       
    # ([eps, …])	Multi-task Lasso model trained with L1/L2 mixed-norm as regularizer.
    
    # (sklearn.linear_model.MultiTaskElasticNetCV, {}),  
    # ([…])	Multi-task L1/L2 ElasticNet with built-in cross-validation.
    
    (sklearn.linear_model.OrthogonalMatchingPursuit, {}),    
    # ([…])	Orthogonal Matching Pursuit model(OMP)
    
    (sklearn.linear_model.OrthogonalMatchingPursuitCV, {}),  
    # ([…])	Cross-validated Orthogonal Matching Pursuit model (OMP).
    
    # (sklearn.linear_model.PassiveAggressiveClassifier, {}),  
    # ([…])	Passive Aggressive Classifier
    
    (sklearn.linear_model.PassiveAggressiveRegressor, {}),   
    # ([C, …])	Passive Aggressive Regressor # Poor Score
    
    # (sklearn.linear_model.Perceptron, {}),             
    # ([penalty, alpha, …])	Read more in the User Guide.
    
    (sklearn.linear_model.RANSACRegressor, {}),        
    # ([…])	RANSAC (RANdom SAmple Consensus) algorithm.  # Poor Score
    
    (sklearn.linear_model.Ridge, {}),                  
    # ([alpha, fit_intercept, …])	Linear least squares with l2 regularization.
    
    # (sklearn.linear_model.RidgeClassifier, {}),        
    # ([alpha, …])	Classifier using Ridge regression.
    
    # (sklearn.linear_model.RidgeClassifierCV, {}),      
    # ([alphas, …])	Ridge classifier with built-in cross-validation.
    
    (sklearn.linear_model.RidgeCV, {}),                
    # ([alphas, …])	Ridge regression with built-in cross-validation.
    
    # (sklearn.linear_model.SGDClassifier, {}),          
    # ([loss, penalty, …])	Linear classifiers(SVM, logistic regression, a.o.) with SGD training.
    
    (sklearn.linear_model.SGDRegressor, {}),           
    # ([loss, penalty, …])	Linear model fitted by minimizing a regularized empirical loss with SGD
    
    # (sklearn.linear_model.TheilSenRegressor, {}),      
    # ([…])	Theil-Sen Estimator: robust multivariate regression model.  # Cause OOM Exception
    
    # sklearn.linear_model.enet_path(X_train, Y_train),           
    # (X, y[, l1_ratio, …])	Compute elastic net path with coordinate descent
    
    # sklearn.linear_model.lars_path(X_train, Y_train),           
    # (X, y[, Xy, Gram, …])	Compute Least Angle Regression or Lasso path using LARS algorithm [1]
    
    # sklearn.linear_model.lasso_path(X_train, Y_train),          
    # (X, y[, eps, …])	Compute Lasso path with coordinate descent
    
    # sklearn.linear_model.logistic_regression_path(X_train, Y_train),  
    # (X, y)	Compute a Logistic Regression model for a list of regularization parameters.
    
    # sklearn.linear_model.orthogonal_mp(X_train, Y_train),       
    # (X, y[, …])	Orthogonal Matching Pursuit(OMP)
    
    # sklearn.linear_model.orthogonal_m_gram(X_train, Y_train),  
    # (Gram, Xy[, …])	Gram Orthogonal Matching Pursuit(OMP)
    
    # sklearn.linear_model.ridge_regression(X_train, Y_train, alpha=0.1),    
    # (X, y, alpha[, …])	Solve the ridge equation by the method of normal equations.
]

In [ ]:
%%time
import time
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning) 

def fit_predict(model_class, kwargs, verbose=True):
    time_start = time.perf_counter()    

    name  = model_class.__name__
    model = model_class(**kwargs)  
    model.fit(X_train, Y_train)
    rmse       = sklearn.metrics.mean_squared_error(Y_valid, model.predict(X_valid), squared=False)
    prediction = model.predict(X_test)
    
    time_taken = time_start - time.perf_counter()
    if verbose: print(f'{time_taken:3.1f}s | rmse = {rmse:3.3f} | {name}')        
    return name, rmse, prediction 


scores      = {}
predictions = {}
for model_class, kwargs in models:
    try:
        name, rmse, prediction = fit_predict(model_class, kwargs)
        scores[name]      = rmse
        predictions[name] = prediction
    except:
        print('ERROR', model_class.__name__)

In [ ]:
scores = dict(sorted(scores.items(), key=itemgetter(1), reverse=False))
scores

# Ensemble

In [ ]:
Y_test = np.mean([
    predictions[name]
    for name, score in scores.items()
    if score <= 0.8  # Remove poor performers
], axis=0)
print('Y_test.shape', Y_test.shape)

# Submission

In [ ]:
submission_df = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv', index_col='id')
submission_df['target'] = Y_test
submission_df.to_csv('submission.csv')
!head submission.csv

# Further Reading

This notebook is part of a series exploring Tabular Data:

[Titanic](https://www.kaggle.com/competitions/titanic)
- [Profilereport EDA](https://www.kaggle.com/code/jamesmcguigan/titanic-profilereport-eda)

[Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic)
- [Profilereport EDA](https://www.kaggle.com/code/jamesmcguigan/titanic-profilereport-eda)
- 0.69932 - [XGBoost](https://www.kaggle.com/code/jamesmcguigan/spaceship-titanic-xgboost)

[Tabular Playground - Jan 2021](https://www.kaggle.com/c/tabular-playground-series-jan-2021)
- 0.72746 / 0.72935 - [scikit-learn Ensemble](https://www.kaggle.com/jamesmcguigan/tabular-playground-scikit-learn-ensemble)
- 0.71552 / 0.71659 - [Fast.ai Tabular Solver](https://www.kaggle.com/jamesmcguigan/fast-ai-tabular-solver)
- 0.70317 / 0.70426 - [XGBoost](https://www.kaggle.com/jamesmcguigan/tabular-playground-xgboost)
- 0.70011 / 0.70181 - [LightGBM](https://www.kaggle.com/jamesmcguigan/tabular-playground-lightgbm)

[Tabular Playground - Feb 2021](https://www.kaggle.com/c/tabular-playground-series-feb-2021)
- 0.84452 - [PyCaret2 AutoML Regression](https://www.kaggle.com/jamesmcguigan/tps-pycaret2-automl-regression)

[Tabular Playground - May 2022](https://www.kaggle.com/c/tabular-playground-series-feb-2021)
- 0.97134 - [LightGBM + XGBoost + Catboost](https://www.kaggle.com/jamesmcguigan/tps-may-2022-lightgbm-xgboost-catboost)


If you found this notebook useful or learnt something new, then please upvote!